In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [3]:
import os
print(os.listdir("C:/Users/Tsenguunn/Documents/Kaggle"))

['100_datasets.pdf', 'concrete_data.csv', 'Concrete_Strength_Prediction_Using_Keras_Tsenguun_Enkhbaatar_2.ipynb', 'gold_price_time_series_monthly.csv', 'home-credit-default-risk', 'home-credit-default-risk.zip', 'house price', 'property-sales', 'property-sales.zip', 'sentiment labelled sentences', 'sentiment-labelled-sentences-data-set (1).zip', 'zara_data_go_2019_all_dataset', 'zara_data_go_2019_all_dataset_A96QYAu7882jqg4z.zip', '^DJI.csv']


In [4]:
filepath_dict = { 'yelp': 'C:/Users/Tsenguunn/Documents/Kaggle/sentiment labelled sentences/yelp_labelled.txt',
                  'amazon': 'C:/Users/Tsenguunn/Documents/Kaggle/sentiment labelled sentences/amazon_cells_labelled.txt',
                  'imdb': 'C:/Users/Tsenguunn/Documents/Kaggle/sentiment labelled sentences/imdb_labelled.txt' }

In [5]:
df_list = []

for source, filepath in filepath_dict.items():
    df = pd.read_csv( filepath, names=['sentences', 'label'], sep='\t' )  
    df['source'] = source
    
    df_list.append(df)

In [6]:
df = pd.concat(df_list)

<b> <font size="4">  Here, I perform the classification prediction on the customer review datasets: Yelp, IMDB, and Amazon. The model classifies a customer review as the positive review or the negative reveiw. I build the Recurrent Neural Network model in PyTorch. The type of the Recurrent Neural Network model is the LSTM model. </font> </b>

In [7]:
# df_yelp = df[df['source'] == 'yelp']
# sentences = df_yelp['sentences'].values
# y = df_yelp['label'].values

In [8]:
sentences = df['sentences'].values

In [9]:
y = df['label'].values

In [21]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [22]:
sentences_train[0]

"I'd rather eat airline food, seriously."

In [23]:
sentences_test.shape

(687,)

In [24]:
df.shape

(2748, 3)

In [26]:
import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tsenguunn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
words = Counter()

<b> <font size="4">   Here, I tokenize sentences as a vector of words. This way, the model treats a sentence as the sequence of words. </font> </b>

In [18]:
#del sentences_train

In [28]:
for i, sentence in enumerate(sentences_train):
    # The sentences will be stored as a list of words/tokens
    sentences_train[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        sentences_train[i].append(word)
    #if i%20000 == 0:
    #    print(str((i*100)/num_train) + "% done")
print("100% done")

100% done


In [29]:
sentences_train[0]

['I', "'d", 'rather', 'eat', 'airline', 'food', ',', 'seriously', '.']

In [30]:
words.items()

dict_items([('i', 807), ("'d", 20), ('rather', 12), ('eat', 17), ('airline', 1), ('food', 88), (',', 1044), ('seriously', 10), ('.', 2050), ('there', 91), ('was', 448), ('a', 697), ('warm', 5), ('feeling', 11), ('with', 222), ('the', 1544), ('service', 77), ('and', 871), ('felt', 10), ('like', 98), ('their', 37), ('guest', 1), ('for', 259), ('special', 12), ('treat', 2), ('fantastic', 15), ('earphones', 1), ('we', 74), ('started', 12), ('tuna', 2), ('sashimi', 3), ('which', 45), ('brownish', 1), ('in', 322), ('color', 5), ('obviously', 6), ("n't", 218), ('fresh', 12), ('great', 165), ('phone', 131), ('!', 373), ('highly', 19), ('recommended', 9), ('a+', 1), ('jabra', 5), ('eargels', 1), ('fit', 11), ('my', 191), ('ears', 5), ('very', 188), ('well', 64), ('do', 101), ('buy', 14), ('this', 508), ('product', 41), ('you', 168), ('wo', 20), ('forget', 3), ('movie', 146), ('rating', 4), (':', 29), ('0/10', 2), ('(', 76), ('grade', 1), ('z', 1), (')', 82), ('note', 4), ('show', 16), ('is', 58

In [31]:
num_train = sentences_train.shape[0]
num_test = sentences_test.shape[0]

In [32]:
# Removing the words that only appear once
words = {k:v for k,v in words.items() if v>1}

In [33]:
words.items()

dict_items([('i', 807), ("'d", 20), ('rather', 12), ('eat', 17), ('food', 88), (',', 1044), ('seriously', 10), ('.', 2050), ('there', 91), ('was', 448), ('a', 697), ('warm', 5), ('feeling', 11), ('with', 222), ('the', 1544), ('service', 77), ('and', 871), ('felt', 10), ('like', 98), ('their', 37), ('for', 259), ('special', 12), ('treat', 2), ('fantastic', 15), ('we', 74), ('started', 12), ('tuna', 2), ('sashimi', 3), ('which', 45), ('in', 322), ('color', 5), ('obviously', 6), ("n't", 218), ('fresh', 12), ('great', 165), ('phone', 131), ('!', 373), ('highly', 19), ('recommended', 9), ('jabra', 5), ('fit', 11), ('my', 191), ('ears', 5), ('very', 188), ('well', 64), ('do', 101), ('buy', 14), ('this', 508), ('product', 41), ('you', 168), ('wo', 20), ('forget', 3), ('movie', 146), ('rating', 4), (':', 29), ('0/10', 2), ('(', 76), (')', 82), ('note', 4), ('show', 16), ('is', 588), ('so', 122), ('bad', 75), ('that', 241), ('even', 63), ('mother', 3), ('of', 494), ('cast', 14), ('pull', 2), ('

<b> <font size="4"> Here, I create the dictionary for words in the 3 datasets. </font> </b>

In [34]:
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)

In [35]:
# Adding padding and unknown to our vocabulary so that they will be assigned an index
words = ['_PAD','_UNK'] + words

In [36]:
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

<b> <font size="4"> Here, I create word to index and index to word conversion libraries. </font> </b>

In [37]:
idx2word.items()

dict_items([(0, '_PAD'), (1, '_UNK'), (2, '.'), (3, 'the'), (4, ','), (5, 'and'), (6, 'i'), (7, 'a'), (8, 'it'), (9, 'is'), (10, 'to'), (11, 'this'), (12, 'of'), (13, 'was'), (14, '!'), (15, 'in'), (16, 'for'), (17, 'not'), (18, 'that'), (19, 'with'), (20, "n't"), (21, "'s"), (22, 'my'), (23, 'very'), (24, 'you'), (25, 'on'), (26, 'great'), (27, 'good'), (28, 'have'), (29, 'but'), (30, 'movie'), (31, 'are'), (32, '0'), (33, 'as'), (34, 'phone'), (35, 'film'), (36, '1'), (37, 'so'), (38, 'be'), (39, 'all'), (40, 'one'), (41, 'had'), (42, 'do'), (43, 'at'), (44, 'like'), (45, 'there'), (46, 'just'), (47, 'food'), (48, 'were'), (49, 'they'), (50, 'time'), (51, ')'), (52, 'from'), (53, 'really'), (54, 'if'), (55, 'place'), (56, 'service'), (57, 'an'), (58, '('), (59, 'would'), (60, '...'), (61, 'bad'), (62, 'we'), (63, 'did'), (64, 'out'), (65, 'no'), (66, 'about'), (67, 'or'), (68, '-'), (69, 'best'), (70, 'ever'), (71, 'well'), (72, 'even'), (73, 'by'), (74, 'has'), (75, 'only'), (76, 'h

In [38]:
for i, sentence in enumerate(sentences_train):
    # Looking up the mapping dictionary and assigning the index to the respective words
    sentences_train[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

In [39]:
for i, sentence in enumerate(sentences_test):
    # For test sentences, we have to tokenize the sentences as well
    sentences_test[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]

In [40]:
sentences_train[1:11]

array([list([0, 13, 7, 607, 329, 19, 3, 56, 5, 0, 357, 44, 121, 0, 16, 7, 305, 1213, 2]),
       list([0, 0, 2]),
       list([0, 306, 19, 3, 1214, 905, 98, 13, 0, 15, 608, 5, 537, 13, 20, 307, 2]),
       list([0, 34, 14, 2]), list([0, 392, 0]),
       list([0, 0, 0, 330, 22, 610, 23, 71, 2]),
       list([0, 20, 266, 11, 110, 2]),
       list([0, 198, 20, 906, 11, 30, 14]),
       list([0, 145, 1215, 58, 0, 145, 0, 51, 0, 145, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]),
       list([0, 1217, 43, 3, 538, 393, 10, 225, 189, 7, 722, 909, 0, 0, 188, 308, 2])],
      dtype=object)

In [41]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

<b> <font size="4"> Here, I add padding to short sentences to create equal length sentences. </font> </b>

In [42]:
seq_len = 100  # The length that the sentences will be padded/shortened to


In [43]:
#del sentences_train
#del sentences_test
sentences_train = pad_input(sentences_train, seq_len)
sentences_test = pad_input(sentences_test, seq_len)

In [44]:
y_train = np.array( y_train )
y_test = np.array( y_test )

In [45]:
sentences_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 197, 304, 224,   0,  47,   4, 356,   2])

In [46]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn


In [47]:
train_data = TensorDataset(torch.from_numpy(sentences_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(sentences_test), torch.from_numpy(y_test))

In [122]:
batch_size = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [158]:
class Sent_Ana(nn.Module):
    
    def __init__( self, vocab_size, embedding_dim, hidden_dim, output_size, n_layers, seq_len, drop_prob = 0.2 ):
        super(Sent_Ana, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding( vocab_size, embedding_dim )
        self.lstm = nn.LSTM( embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first=True )
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear( hidden_dim, output_size )
        self.sigmoid = nn.Sigmoid()
        #self.fc2 = nn.Linear( hidden_dim2, output_size )
        
    def forward(self, x):
        #batch_size = x.size(0)
        x = x.long()
        x = self.embedding(x)
        #x = self.lstm( x )
        x, states = self.lstm(x)
        x = x.contiguous().view(-1, self.hidden_dim)
        x = self.dropout(x)
        x = self.fc( x )
        #x = F.relu( x )
        #x = self.fc2( x )
        x = F.sigmoid( x )
        x = x.view(batch_size, -1)
        x = x[:,-1]
        #x = x.view(batch_size, -1)
        return x
    
    
        
        
        
        
        

<b> <font size="4"> Here, I build the Deep Learning model with Embedding layer, 2 LSTM layers, and one Dense layer.
    Here are the parameters:
    <br>
    Batch size - 32 <br>
    Embedding vector size - 250 <br>
    Dense layer units number - 32 <br>
    Number of epochs - 20
    </font> </b>

In [167]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 250
hidden_dim = 32
n_layers = 2
hidden_dim2 = 16

In [168]:
#model = Sent_Ana( vocab_size, embedding_dim, hidden_dim, hidden_dim2, output_size, n_layers, seq_len, drop_prob = 0.2 )
model = Sent_Ana( vocab_size, embedding_dim, hidden_dim,  output_size, n_layers, seq_len, drop_prob = 0.2 )



In [169]:
model.train()

Sent_Ana(
  (embedding): Embedding(1881, 250)
  (lstm): LSTM(250, 32, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

<b> <font size="4"> Here, I use the Adam optimization algorithm and the Binary Cross Entropy loss function. </font> </b>

In [170]:
optimizer_fit = torch.optim.Adam(model.parameters(), lr=0.01)

In [171]:
criterion = torch.nn.BCELoss()

In [172]:
LOSS = {}
LOSS['training data'] = []
LOSS['validation data'] = []

In [173]:
epochs = 20

def train_model(epochs):
    
    for epoch in range(epochs):
        for x, y in train_loader:
            y = torch.tensor(y, dtype=torch.float)
            optimizer_fit.zero_grad()
            yhat = model(x)
            #print( y.shape )
            #print( yhat.shape )
            #print( x.shape )
            loss = criterion(yhat, y)
            optimizer_fit.zero_grad()
            loss.backward()
            optimizer_fit.step()
        
        print('Epoch: {} and Loss: {}'.format(epoch, loss.item()))
        LOSS['training data'].append(loss.item())
 
        
train_model(epochs)

C:\Users\Tsenguunn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Epoch: 0 and Loss: 0.5510993003845215
Epoch: 1 and Loss: 0.5142314434051514
Epoch: 2 and Loss: 0.48300111293792725
Epoch: 3 and Loss: 0.28951066732406616
Epoch: 4 and Loss: 0.24134567379951477
Epoch: 5 and Loss: 0.09960436075925827
Epoch: 6 and Loss: 0.08615749329328537
Epoch: 7 and Loss: 0.030248187482357025
Epoch: 8 and Loss: 0.25190865993499756
Epoch: 9 and Loss: 0.038869161158800125
Epoch: 10 and Loss: 0.023376772180199623
Epoch: 11 and Loss: 0.039004627615213394
Epoch: 12 and Loss: 0.12242406606674194
Epoch: 13 and Loss: 0.022920528426766396
Epoch: 14 and Loss: 0.0044870516285300255
Epoch: 15 and Loss: 0.08641945570707321
Epoch: 16 and Loss: 0.05490243434906006
Epoch: 17 and Loss: 0.1387859731912613
Epoch: 18 and Loss: 0.09814427047967911
Epoch: 19 and Loss: 0.0273975171148777


In [119]:
model.eval()

Sent_Ana(
  (embedding): Embedding(1881, 100)
  (lstm): LSTM(100, 16, num_layers=4, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

model(test_data[)

In [130]:
#torch.from_numpy(sentences_test[0])

In [236]:
#model(torch.from_numpy(sentences_test[0]).view(-1,1))

In [174]:
model.eval()
num_correct = 0

for x, y in test_loader:
    
    y = torch.tensor(y, dtype=torch.float)
    optimizer_fit.zero_grad()
    yhat = model(x)
    loss = criterion(yhat, y)
    LOSS['validation data'].append(loss.item())
    pred = torch.round(yhat.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(y.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(LOSS['validation data'])))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))
    
    

C:\Users\Tsenguunn\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Test loss: 0.977
Test accuracy: 76.274%


<b> <font size="4"> We can see that the accuracy on the test data is 76.3%. This means that actually the Convolutional Neural Network model with the Embedding layer performs better than the Recurrent Neural Network model with the Embedding layer in classifying the customer reveiw as the positive review or the negative review. </font> </b>